# Training a CNN classifier with VAE augmented data

### Imports

In [1]:
import mlflow
import torch
import vae
from utils.mlflow import backend_stores
from utils.trainer import TrainingArguments
from vae.generation import Generator, augmentations
from vae.models import VAEConfig
from evaluation.models import CNNMNIST
from evaluation.train_utils import train_model
from utils.data import load_datasets

vae.models.base.model_store = "pretrained_models/MNIST"

### Seeding, loading data & setting up mlflow logging

In [2]:
SEED = 1337
DATASET = "MNIST"
# set the backend store uri of mlflow
mlflow.set_tracking_uri(getattr(backend_stores, DATASET))
# seed torch
torch.manual_seed(SEED)
# load datasets
train_dataset, _, val_dataset, test_dataset = load_datasets(DATASET)

### The parameters for the classification task

In [3]:
training_args = TrainingArguments(
    total_steps=5000,
    batch_size=32,
    validation_intervall=200,
    save_model=False,
    seed=SEED,
    early_stopping=False,
    early_stopping_window=20,
    save_best_metric="best_acc",
)

### Parameters for data augmentation

In [4]:
MULTI_VAE = False
VAE_EPOCHS = 50
Z_DIM = 2
BETA = 1.0
AUGMENTATION = augmentations.REPARAMETRIZATION
augmentation_params = {"K": 450}

## Training the CNN

In [5]:
# create a vae config
vae_config = VAEConfig(z_dim=Z_DIM, beta=BETA)
# start mlflow run in experiment
mlflow.set_experiment(f"CNN Z_DIM {Z_DIM}" if AUGMENTATION else "CNN Baseline")
with mlflow.start_run(run_name=AUGMENTATION or "baseline"):
    # perform data augmentation if specified
    if AUGMENTATION is not None:
        gen = Generator(
            vae_config,
            VAE_EPOCHS,
            MULTI_VAE,
            SEED,
        )
        train_dataset = gen.augment_dataset(train_dataset, AUGMENTATION, **augmentation_params)

    # train cnn
    results = train_model(
        model=CNNMNIST(),
        training_args=training_args,
        train_dataset=train_dataset,
        dev_dataset=val_dataset,
        test_dataset=test_dataset,
        seed=SEED,
    )
    # print the results
    print(results)

INFO: 'CNN Z_DIM 2' does not exist. Creating a new experiment


classes: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
class_counts: tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5]), (50 total)
n = 10
x = tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000])
L = tensor([45, 45, 45, 45, 45, 45, 45, 45, 45, 45])
----------------------------------------
class_counts: tensor([45, 45, 45, 45, 45, 45, 45, 45, 45, 45]), (450 total)
x = tensor([0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
        0.1000])
----------------------------------------
Augmented dataset from 50 samples to 500
----------------------------------------
class_counts: tensor([50, 50, 50, 50, 50, 50, 50, 50, 50, 50]), (500 total)
x = tensor([0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111, 0.1111,
        0.1111])


Training epoch 313: 100%|██████████| 5000/5000 [02:06<00:00, 39.65it/s, loss=0.16]

Evaluating model.


{'eval_loss': 1.4287031185150147, 'eval_acc': 0.7242, 'eval_f1': 0.7179893304101919, 'eval_best_acc': 0.7242}
